In [ ]:
!pip install transformers accelerate einops Xformers langchain InstructorEmbedding sentence-transformers chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install unstructured pandoc

In [ ]:
# CLEAR CUDA
import gc
torch.cuda.empty_cache()
gc.collect()


In [ ]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer

model_name = "databricks/dolly-v2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

generate_text = pipeline(model=model_name, torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)

In [ ]:
hf_pipeline

In [ ]:
# TODO: Check if input is checked somewhere, otherwise check something like this:

# Your output sequence
output_sequence = "This is an example output sequence."

# Tokenize the output sequence
tokenized_output = tokenizer.encode(output_sequence, return_tensors="pt")

# Count the number of tokens in the tokenized sequence
num_tokens = tokenized_output.shape[1]

print(f"Number of tokens in the output sequence: {num_tokens}")

# Test LLM Chain

In [ ]:
question = 'Who was Dolly the sheep?'
llm_chain.run(question)

In [ ]:
question = 'what are the colors of the rainbow?'
llm_chain.run(question)

# Prepare Documents


In [ ]:
!pip install nest_asyncio

In [ ]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from langchain.document_loaders.sitemap import SitemapLoader


In [ ]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
sitemap_loader = SitemapLoader(web_path="https://www.limestone.on.ca/sitemap.xml")

docs = sitemap_loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [ ]:
len(texts)

In [ ]:
texts[2]

# Download HuggingFace Embeddings
Check [MTEB English Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) to make sure you download embeddings with good performance

In [ ]:
# Choose one of the top performers from the MTEB English Leaderboard

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# top #2 when task = Retrieval June 2023 for under ~500 MB
model_name = "intfloat/e5-base-v2" 

hf = HuggingFaceEmbeddings(model_name=model_name)

# Make a Vector Database

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = hf 

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# and a retriever

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={'k':2})

# Test the basic retriever

In [ ]:
docs = retriever.get_relevant_documents("How much is tuition?")

In [ ]:
docs[0]

# Make a proper Question Retrieval chain

In [ ]:
from langchain.memory import ConversationBufferMemory

from typing import Dict, Any

class AnswerConversationBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerConversationBufferMemory, self).save_context(inputs,{'response': outputs['result']})

In [ ]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                                   chain_type="stuff", 
                                                   retriever=retriever, 
                                                   return_source_documents=True,
                                                   memory=AnswerConversationBufferMemory())

In [ ]:
# try to set the tone to "as a human-like Student Advisor"
qa_chain_with_memory.combine_documents_chain.llm_chain.prompt.template = '''
You are a prospective student advisor for a university. You are talkative and provide lots of specific details using the context given by the users question. 
If you don't know the answer, you truthfully say you don't know and don't try to make up an answer. You use the information contained in the "Relevant Information" section and You always answer with unbiased, ETHICAL and safe advise.

Relevant Information:
{history}

Conversation:

Question: {question}
Helpful Answer:'''

prompt = PromptTemplate(
    input_variables=["history", "question"], template=template)

In [ ]:
query = "How much is tuition in engineering?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "How much is tuition in med school?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "I am interested in both medicine and engineering, do you have a program that is a mix of the two?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)

# Trying out Knowledge Graph Memory

In [ ]:
# workaround again
class AnswerKnowledgeGraphMemory(ConversationKGMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerKnowledgeGraphMemory, self).save_context(inputs,{'response': outputs['result']})

In [ ]:
from langchain.memory import ConversationKGMemory

memory = AnswerKnowledgeGraphMemory(llm=hf_pipeline)

qa_chain_with_kg_memory = RetrievalQA.from_chain_type(llm=hf_pipeline,
                                                      chain_type="stuff", 
                                                      retriever=retriever, 
                                                      return_source_documents=True,                                                      
                                                      memory=memory)

In [ ]:
# try to set the tone to "as a human-like Student Advisor"
template = '''
You are a prospective student advisor for a university. You are talkative and provide lots of specific details using the context given by the users question. 
If you don't know the answer, you truthfully say you don't know and don't try to make up an answer. You use the information contained in the "Relevant Information" section and You always answer with unbiased, ETHICAL and safe advise.

Relevant Information:
{history}

Conversation:

Question: {question}
Helpful Answer:'''

prompt = PromptTemplate(
    input_variables=["history", "question"], template=template)

In [ ]:
qa_chain_with_kg_memory.combine_documents_chain.llm_chain.prompt = prompt
qa_chain_with_kg_memory.combine_documents_chain.llm_chain.prompt

In [ ]:
qa_chain_with_kg_memory.combine_documents_chain

In [ ]:
query = "I am interested in both medicine and engineering, do you have a program that is a mix of the two?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "I like biomedical engineering, do you offer it as a graduate program?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "How much is tuition for this program?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

In [ ]:
query = "WOw, expensive! is this the fee for international students?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

# Notes for next version
- Make a National Building Code chatbot 
- Make a school board chatbot
- See if 7B model can be handled by free DataBricks
- Figure out how to handle data in databricks using dbfs ([Dolly Example](https://www.dbdemos.ai/demo-notebooks.html?demoName=llm-dolly-chatbot))